# Imports

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import scipy as sp
from scipy import stats
import numpy as np
import pandas as pd

import statistics

# Load Data

In [2]:
demo_df = statistics.load_demographic_features()
statistics.compute_demo_features(demo_df)
tracing_df = statistics.load_tracing_features(sort='order')
recording_df = statistics.load_recording_relations()
recording_df = statistics.associate_recordings(recording_df, tracing_df, demo_df)
gaze_dfs = statistics.load_gaze_features()
statistics.check_gaze_recording_associations(recording_df, gaze_dfs)
statistics.compute_gaze_features(gaze_dfs)
gaze_df = statistics.combine_gaze_features(gaze_dfs)
full_df = statistics.combine_all_features(recording_df, gaze_df)
full_df.columns

Index(['id', 'subjectNumber', 'scenarioNumber', 'newAfterOld', 'scenarioType',
       'displayType', 'sensorPlacementTime', 'ppvStartTime', 'ccStartTime',
       'inSpO2TargetRangeDuration', 'inSpO2LooseTargetRangeDuration',
       'inSpO2TargetRangeStartTime', 'aboveSpO2TargetRangeDuration',
       'belowSpO2TargetRangeDuration', 'inFiO2TargetRangeDuration',
       'inFiO2TargetRangeStartTime', 'aboveFiO2TargetRangeDuration',
       'belowFiO2TargetRangeDuration', 'spO2SignedErrorIntegral',
       'spO2UnsignedErrorIntegral', 'spO2SquaredErrorIntegral',
       'fiO2LargeAdjustments', 'code', 'education', 'cumulativeNICU',
       'resusLD', 'resusLDnonzero', 'cumulativeNICU5', 'cumulativeNICU10',
       'cumulativeNICU13', 'resusLD20', 'visitDuration_fiO2Dial',
       'visitDuration_infant', 'visitDuration_monitorApgarTimer',
       'visitDuration_monitorBlank', 'visitDuration_monitorFiO2',
       'visitDuration_monitorFull', 'visitDuration_monitorGraph',
       'visitDuration_monitorH

### Scenario Type Split

In [3]:
scenario_split = statistics.build_pairing(
    full_df, 'scenarioType',
    intersect_subjects=False, check_validity=False
)
scenario_split.describe()

Pairing against scenarioType:
  0: easy vs. 1: hard
  30 0 examples vs. 34 1 examples.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


## Led resuscitations in L&D in past year?

In [4]:
resusLD_pairing = statistics.build_pairing(
    full_df, 'resusLDnonzero',
    intersect_subjects=False, check_validity=False
)
resusLD_pairing.describe()

Pairing against resusLDnonzero:
  0: noLDResusPastYear vs. 1: LDResusPastYear
  32 0 examples vs. 28 1 examples.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


### Not Led, Easy Scenarios

In [5]:
resusLD_easy_pairing = statistics.build_pairing(
    scenario_split[0], 'resusLDnonzero',
    intersect_subjects=False, check_validity=False
)
resusLD_easy_pairing.describe()

Pairing against resusLDnonzero:
  0: noLDResusPastYear vs. 1: LDResusPastYear
  16 0 examples vs. 12 1 examples.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [6]:
display_pairing = statistics.build_pairing(resusLD_easy_pairing[0], 'displayType')
display_pairing.describe()

Pairing against displayType:
  0: minimal vs. 1: full
  8 0 vs. 1 pairs.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [7]:
statistics.test_gaze_duration_outcomes(display_pairing)

visitDuration_infant:
  mean diff = -0.537; stdev diff = 17.249
  Paired t-test:
    |diff| > 0: p = 0.937
    diff < 0: p = 0.468
    diff > 0: p = 0.532
visitDuration_warmerInstrumentPanel:
  mean diff = -0.372; stdev diff = 9.440
  Paired t-test:
    |diff| > 0: p = 0.920
    diff < 0: p = 0.460
    diff > 0: p = 0.540
visitDuration_fiO2Dial:
  mean diff = -0.292; stdev diff = 5.954
  Paired t-test:
    |diff| > 0: p = 0.900
    diff < 0: p = 0.450
    diff > 0: p = 0.550
visitDuration_spO2ReferenceTable:
  mean diff = -1.617; stdev diff = 3.801
  Paired t-test:
    |diff| > 0: p = 0.297
   ~diff < 0: p = 0.149
    diff > 0: p = 0.851
visitDuration_monitorFull:
  mean diff = -6.360; stdev diff = 37.325
  Paired t-test:
    |diff| > 0: p = 0.666
    diff < 0: p = 0.333
    diff > 0: p = 0.667
visitDuration_monitorBlank:
  mean diff = 2.457; stdev diff = 7.199
  Paired t-test:
    |diff| > 0: p = 0.397
    diff < 0: p = 0.802
   ~diff > 0: p = 0.198
visitDuration_monitorApgarTimer:
  

### Not Led, Hard Scenarios

In [8]:
resusLD_hard_pairing = statistics.build_pairing(
    scenario_split[1], 'resusLDnonzero',
    intersect_subjects=False, check_validity=False
)
resusLD_hard_pairing.describe()

Pairing against resusLDnonzero:
  0: noLDResusPastYear vs. 1: LDResusPastYear
  16 0 vs. 1 pairs.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [9]:
display_pairing = statistics.build_pairing(resusLD_hard_pairing[0], 'displayType')
display_pairing.describe()

Pairing against displayType:
  0: minimal vs. 1: full
  8 0 vs. 1 pairs.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [10]:
statistics.test_gaze_duration_outcomes(display_pairing)

visitDuration_infant:
  mean diff = -6.231; stdev diff = 27.362
  Paired t-test:
    |diff| > 0: p = 0.566
    diff < 0: p = 0.283
    diff > 0: p = 0.717
visitDuration_warmerInstrumentPanel:
  mean diff = 1.974; stdev diff = 8.942
  Paired t-test:
    |diff| > 0: p = 0.577
    diff < 0: p = 0.711
    diff > 0: p = 0.289
visitDuration_fiO2Dial:
  mean diff = 0.754; stdev diff = 3.369
  Paired t-test:
    |diff| > 0: p = 0.573
    diff < 0: p = 0.714
    diff > 0: p = 0.286
visitDuration_spO2ReferenceTable:
  mean diff = 0.645; stdev diff = 2.321
  Paired t-test:
    |diff| > 0: p = 0.486
    diff < 0: p = 0.757
    diff > 0: p = 0.243
visitDuration_monitorFull:
  mean diff = -2.823; stdev diff = 18.924
  Paired t-test:
    |diff| > 0: p = 0.705
    diff < 0: p = 0.352
    diff > 0: p = 0.648
visitDuration_monitorBlank:
  mean diff = 9.211; stdev diff = 11.651
  Paired t-test:
   *|diff| > 0: p = 0.075
    diff < 0: p = 0.963
  **diff > 0: p = 0.037
visitDuration_monitorApgarTimer:
  me

### Led, Easy Scenarios

In [11]:
resusLD_easy_pairing = statistics.build_pairing(
    scenario_split[0], 'resusLDnonzero',
    intersect_subjects=False, check_validity=False
)
resusLD_easy_pairing.describe()

Pairing against resusLDnonzero:
  0: noLDResusPastYear vs. 1: LDResusPastYear
  16 0 examples vs. 12 1 examples.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [12]:
display_pairing = statistics.build_pairing(resusLD_easy_pairing[1], 'displayType')
display_pairing.describe()

Pairing against displayType:
  0: minimal vs. 1: full
  6 0 vs. 1 pairs.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [13]:
statistics.test_gaze_duration_outcomes(display_pairing)

visitDuration_infant:
  mean diff = -10.121; stdev diff = 18.241
  Paired t-test:
    |diff| > 0: p = 0.270
   ~diff < 0: p = 0.135
    diff > 0: p = 0.865
visitDuration_warmerInstrumentPanel:
  mean diff = -0.950; stdev diff = 4.813
  Paired t-test:
    |diff| > 0: p = 0.677
    diff < 0: p = 0.339
    diff > 0: p = 0.661
visitDuration_fiO2Dial:
  mean diff = 3.439; stdev diff = 6.508
  Paired t-test:
    |diff| > 0: p = 0.290
    diff < 0: p = 0.855
   ~diff > 0: p = 0.145
visitDuration_spO2ReferenceTable:
  mean diff = 3.799; stdev diff = 7.142
  Paired t-test:
    |diff| > 0: p = 0.288
    diff < 0: p = 0.856
   ~diff > 0: p = 0.144
visitDuration_monitorFull:
  mean diff = -0.712; stdev diff = 28.850
  Paired t-test:
    |diff| > 0: p = 0.958
    diff < 0: p = 0.479
    diff > 0: p = 0.521
visitDuration_monitorBlank:
  mean diff = 5.885; stdev diff = 3.921
  Paired t-test:
  **|diff| > 0: p = 0.020
    diff < 0: p = 0.990
  **diff > 0: p = 0.010
visitDuration_monitorApgarTimer:
  m

### Led, Hard Scenarios

In [14]:
resusLD_hard_pairing = statistics.build_pairing(
    scenario_split[1], 'resusLDnonzero',
    intersect_subjects=False, check_validity=False
)
resusLD_hard_pairing.describe()

Pairing against resusLDnonzero:
  0: noLDResusPastYear vs. 1: LDResusPastYear
  16 0 vs. 1 pairs.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [15]:
display_pairing = statistics.build_pairing(resusLD_hard_pairing[1], 'displayType')
display_pairing.describe()

Pairing against displayType:
  0: minimal vs. 1: full
  8 0 vs. 1 pairs.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [16]:
statistics.test_gaze_duration_outcomes(display_pairing)

visitDuration_infant:
  mean diff = 17.351; stdev diff = 19.191
  Paired t-test:
  **|diff| > 0: p = 0.048
    diff < 0: p = 0.976
  **diff > 0: p = 0.024
visitDuration_warmerInstrumentPanel:
  mean diff = -4.164; stdev diff = 8.813
  Paired t-test:
    |diff| > 0: p = 0.251
   ~diff < 0: p = 0.126
    diff > 0: p = 0.874
visitDuration_fiO2Dial:
  mean diff = 0.510; stdev diff = 2.734
  Paired t-test:
    |diff| > 0: p = 0.637
    diff < 0: p = 0.682
    diff > 0: p = 0.318
visitDuration_spO2ReferenceTable:
  mean diff = 0.240; stdev diff = 2.688
  Paired t-test:
    |diff| > 0: p = 0.820
    diff < 0: p = 0.590
    diff > 0: p = 0.410
visitDuration_monitorFull:
  mean diff = -19.652; stdev diff = 20.319
  Paired t-test:
  **|diff| > 0: p = 0.038
  **diff < 0: p = 0.019
    diff > 0: p = 0.981
visitDuration_monitorBlank:
  mean diff = -2.157; stdev diff = 7.230
  Paired t-test:
    |diff| > 0: p = 0.456
    diff < 0: p = 0.228
    diff > 0: p = 0.772
visitDuration_monitorApgarTimer:
  

## At least 10 years NICU experience?

In [17]:
cumulativeNICU10_pairing = statistics.build_pairing(
    full_df, 'cumulativeNICU10',
    intersect_subjects=False, check_validity=False
)
cumulativeNICU10_pairing.describe()

Pairing against cumulativeNICU10:
  0: <10CumulativeNICUYears vs. 1: >=10CumulativeNICUYears
  22 0 examples vs. 38 1 examples.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


### Less than 10 years, Easy Scenarios

In [18]:
cumulativeNICU10_easy_pairing = statistics.build_pairing(
    scenario_split[0], 'cumulativeNICU10',
    intersect_subjects=False, check_validity=False
)
cumulativeNICU10_easy_pairing.describe()

Pairing against cumulativeNICU10:
  0: <10CumulativeNICUYears vs. 1: >=10CumulativeNICUYears
  10 0 examples vs. 18 1 examples.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [19]:
display_pairing = statistics.build_pairing(cumulativeNICU10_easy_pairing[0], 'displayType')
display_pairing.describe()

Pairing against displayType:
  0: minimal vs. 1: full
  5 0 vs. 1 pairs.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [20]:
statistics.test_gaze_duration_outcomes(display_pairing)

visitDuration_infant:
  mean diff = -5.379; stdev diff = 21.295
  Paired t-test:
    |diff| > 0: p = 0.640
    diff < 0: p = 0.320
    diff > 0: p = 0.680
visitDuration_warmerInstrumentPanel:
  mean diff = -0.724; stdev diff = 5.602
  Paired t-test:
    |diff| > 0: p = 0.809
    diff < 0: p = 0.404
    diff > 0: p = 0.596
visitDuration_fiO2Dial:
  mean diff = 2.823; stdev diff = 7.128
  Paired t-test:
    |diff| > 0: p = 0.473
    diff < 0: p = 0.764
    diff > 0: p = 0.236
visitDuration_spO2ReferenceTable:
  mean diff = 2.667; stdev diff = 5.724
  Paired t-test:
    |diff| > 0: p = 0.404
    diff < 0: p = 0.798
    diff > 0: p = 0.202
visitDuration_monitorFull:
  mean diff = -8.196; stdev diff = 45.814
  Paired t-test:
    |diff| > 0: p = 0.739
    diff < 0: p = 0.369
    diff > 0: p = 0.631
visitDuration_monitorBlank:
  mean diff = 8.382; stdev diff = 1.703
  Paired t-test:
  **|diff| > 0: p = 0.001
    diff < 0: p = 1.000
  **diff > 0: p = 0.000
visitDuration_monitorApgarTimer:
  me

### Less than 10 years, Hard Scenarios

In [21]:
cumulativeNICU10_hard_pairing = statistics.build_pairing(
    scenario_split[1], 'cumulativeNICU10',
    intersect_subjects=False, check_validity=False
)
cumulativeNICU10_hard_pairing.describe()

Pairing against cumulativeNICU10:
  0: <10CumulativeNICUYears vs. 1: >=10CumulativeNICUYears
  12 0 examples vs. 20 1 examples.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [22]:
display_pairing = statistics.build_pairing(cumulativeNICU10_hard_pairing[0], 'displayType')
display_pairing.describe()

Pairing against displayType:
  0: minimal vs. 1: full
  6 0 vs. 1 pairs.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [23]:
statistics.test_gaze_duration_outcomes(display_pairing)

visitDuration_infant:
  mean diff = 25.127; stdev diff = 19.760
  Paired t-test:
  **|diff| > 0: p = 0.036
    diff < 0: p = 0.982
  **diff > 0: p = 0.018
visitDuration_warmerInstrumentPanel:
  mean diff = -5.678; stdev diff = 7.843
  Paired t-test:
   ~|diff| > 0: p = 0.166
   *diff < 0: p = 0.083
    diff > 0: p = 0.917
visitDuration_fiO2Dial:
  mean diff = -2.029; stdev diff = 1.540
  Paired t-test:
  **|diff| > 0: p = 0.032
  **diff < 0: p = 0.016
    diff > 0: p = 0.984
visitDuration_spO2ReferenceTable:
  mean diff = 0.450; stdev diff = 1.593
  Paired t-test:
    |diff| > 0: p = 0.556
    diff < 0: p = 0.722
    diff > 0: p = 0.278
visitDuration_monitorFull:
  mean diff = -24.170; stdev diff = 23.512
  Paired t-test:
   *|diff| > 0: p = 0.070
  **diff < 0: p = 0.035
    diff > 0: p = 0.965
visitDuration_monitorBlank:
  mean diff = 6.722; stdev diff = 15.887
  Paired t-test:
    |diff| > 0: p = 0.388
    diff < 0: p = 0.806
   ~diff > 0: p = 0.194
visitDuration_monitorApgarTimer:
 

### At least 10 years, Easy Scenarios

In [24]:
cumulativeNICU10_easy_pairing = statistics.build_pairing(
    scenario_split[0], 'cumulativeNICU10',
    intersect_subjects=False, check_validity=False
)
cumulativeNICU10_easy_pairing.describe()

Pairing against cumulativeNICU10:
  0: <10CumulativeNICUYears vs. 1: >=10CumulativeNICUYears
  10 0 examples vs. 18 1 examples.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [25]:
display_pairing = statistics.build_pairing(cumulativeNICU10_easy_pairing[1], 'displayType')
display_pairing.describe()

Pairing against displayType:
  0: minimal vs. 1: full
  9 0 vs. 1 pairs.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [26]:
statistics.test_gaze_duration_outcomes(display_pairing)

visitDuration_infant:
  mean diff = -4.237; stdev diff = 16.397
  Paired t-test:
    |diff| > 0: p = 0.486
    diff < 0: p = 0.243
    diff > 0: p = 0.757
visitDuration_warmerInstrumentPanel:
  mean diff = -0.562; stdev diff = 8.795
  Paired t-test:
    |diff| > 0: p = 0.861
    diff < 0: p = 0.431
    diff > 0: p = 0.569
visitDuration_fiO2Dial:
  mean diff = 0.464; stdev diff = 5.902
  Paired t-test:
    |diff| > 0: p = 0.830
    diff < 0: p = 0.585
    diff > 0: p = 0.415
visitDuration_spO2ReferenceTable:
  mean diff = -0.387; stdev diff = 6.041
  Paired t-test:
    |diff| > 0: p = 0.861
    diff < 0: p = 0.430
    diff > 0: p = 0.570
visitDuration_monitorFull:
  mean diff = -1.574; stdev diff = 24.974
  Paired t-test:
    |diff| > 0: p = 0.863
    diff < 0: p = 0.431
    diff > 0: p = 0.569
visitDuration_monitorBlank:
  mean diff = 1.451; stdev diff = 6.483
  Paired t-test:
    |diff| > 0: p = 0.544
    diff < 0: p = 0.728
    diff > 0: p = 0.272
visitDuration_monitorApgarTimer:
  m

### At least 10 years, Hard Scenarios

In [27]:
cumulativeNICU10_hard_pairing = statistics.build_pairing(
    scenario_split[1], 'cumulativeNICU10',
    intersect_subjects=False, check_validity=False
)
cumulativeNICU10_hard_pairing.describe()

Pairing against cumulativeNICU10:
  0: <10CumulativeNICUYears vs. 1: >=10CumulativeNICUYears
  12 0 examples vs. 20 1 examples.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [28]:
display_pairing = statistics.build_pairing(cumulativeNICU10_hard_pairing[1], 'displayType')
display_pairing.describe()

Pairing against displayType:
  0: minimal vs. 1: full
  10 0 vs. 1 pairs.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [29]:
statistics.test_gaze_duration_outcomes(display_pairing)

visitDuration_infant:
  mean diff = -6.181; stdev diff = 22.675
  Paired t-test:
    |diff| > 0: p = 0.435
    diff < 0: p = 0.217
    diff > 0: p = 0.783
visitDuration_warmerInstrumentPanel:
  mean diff = 1.656; stdev diff = 9.171
  Paired t-test:
    |diff| > 0: p = 0.601
    diff < 0: p = 0.699
    diff > 0: p = 0.301
visitDuration_fiO2Dial:
  mean diff = 2.228; stdev diff = 2.620
  Paired t-test:
  **|diff| > 0: p = 0.031
    diff < 0: p = 0.984
  **diff > 0: p = 0.016
visitDuration_spO2ReferenceTable:
  mean diff = 0.438; stdev diff = 2.938
  Paired t-test:
    |diff| > 0: p = 0.665
    diff < 0: p = 0.667
    diff > 0: p = 0.333
visitDuration_monitorFull:
  mean diff = -3.478; stdev diff = 15.421
  Paired t-test:
    |diff| > 0: p = 0.516
    diff < 0: p = 0.258
    diff > 0: p = 0.742
visitDuration_monitorBlank:
  mean diff = 1.611; stdev diff = 6.393
  Paired t-test:
    |diff| > 0: p = 0.469
    diff < 0: p = 0.765
    diff > 0: p = 0.235
visitDuration_monitorApgarTimer:
  mea

## RN vs. MD
Note that only one MD had not performed a resuscitation in L&D in the past year, and only 3 RNs had performed a resuscitation in L&D in the past year.

In [30]:
education_pairing = statistics.build_pairing(
    full_df, 'education',
    intersect_subjects=False, check_validity=False
)
education_pairing.describe()

Pairing against education:
  0: RN vs. 1: MD
  32 0 examples vs. 28 1 examples.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


### RN, Easy Scenarios

In [31]:
education_easy_pairing = statistics.build_pairing(
    scenario_split[0], 'education',
    intersect_subjects=False, check_validity=False
)
education_easy_pairing.describe()

Pairing against education:
  0: RN vs. 1: MD
  16 0 examples vs. 12 1 examples.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [32]:
display_pairing = statistics.build_pairing(education_easy_pairing[0], 'displayType')
display_pairing.describe()

Pairing against displayType:
  0: minimal vs. 1: full
  8 0 vs. 1 pairs.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [33]:
statistics.test_gaze_duration_outcomes(display_pairing)

visitDuration_infant:
  mean diff = -3.692; stdev diff = 15.440
  Paired t-test:
    |diff| > 0: p = 0.547
    diff < 0: p = 0.274
    diff > 0: p = 0.726
visitDuration_warmerInstrumentPanel:
  mean diff = -0.105; stdev diff = 9.481
  Paired t-test:
    |diff| > 0: p = 0.977
    diff < 0: p = 0.489
    diff > 0: p = 0.511
visitDuration_fiO2Dial:
  mean diff = -0.492; stdev diff = 5.941
  Paired t-test:
    |diff| > 0: p = 0.833
    diff < 0: p = 0.416
    diff > 0: p = 0.584
visitDuration_spO2ReferenceTable:
  mean diff = -2.150; stdev diff = 4.520
  Paired t-test:
    |diff| > 0: p = 0.249
   ~diff < 0: p = 0.124
    diff > 0: p = 0.876
visitDuration_monitorFull:
  mean diff = -6.475; stdev diff = 37.337
  Paired t-test:
    |diff| > 0: p = 0.660
    diff < 0: p = 0.330
    diff > 0: p = 0.670
visitDuration_monitorBlank:
  mean diff = 2.442; stdev diff = 7.187
  Paired t-test:
    |diff| > 0: p = 0.399
    diff < 0: p = 0.801
   ~diff > 0: p = 0.199
visitDuration_monitorApgarTimer:
  

### RN, Hard Scenarios

In [34]:
education_hard_pairing = statistics.build_pairing(
    scenario_split[1], 'education',
    intersect_subjects=False, check_validity=False
)
education_hard_pairing.describe()

Pairing against education:
  0: RN vs. 1: MD
  16 0 vs. 1 pairs.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [35]:
display_pairing = statistics.build_pairing(education_hard_pairing[0], 'displayType')
display_pairing.describe()

Pairing against displayType:
  0: minimal vs. 1: full
  8 0 vs. 1 pairs.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [36]:
statistics.test_gaze_duration_outcomes(display_pairing)

visitDuration_infant:
  mean diff = -9.835; stdev diff = 23.868
  Paired t-test:
    |diff| > 0: p = 0.312
   ~diff < 0: p = 0.156
    diff > 0: p = 0.844
visitDuration_warmerInstrumentPanel:
  mean diff = 0.912; stdev diff = 9.729
  Paired t-test:
    |diff| > 0: p = 0.811
    diff < 0: p = 0.594
    diff > 0: p = 0.406
visitDuration_fiO2Dial:
  mean diff = 1.486; stdev diff = 3.432
  Paired t-test:
    |diff| > 0: p = 0.290
    diff < 0: p = 0.855
   ~diff > 0: p = 0.145
visitDuration_spO2ReferenceTable:
  mean diff = 0.157; stdev diff = 3.036
  Paired t-test:
    |diff| > 0: p = 0.895
    diff < 0: p = 0.553
    diff > 0: p = 0.447
visitDuration_monitorFull:
  mean diff = 1.446; stdev diff = 12.936
  Paired t-test:
    |diff| > 0: p = 0.776
    diff < 0: p = 0.612
    diff > 0: p = 0.388
visitDuration_monitorBlank:
  mean diff = 5.810; stdev diff = 9.374
  Paired t-test:
   ~|diff| > 0: p = 0.145
    diff < 0: p = 0.927
   *diff > 0: p = 0.073
visitDuration_monitorApgarTimer:
  mean

### MD, Easy Scenarios

In [37]:
education_easy_pairing = statistics.build_pairing(
    scenario_split[0], 'education',
    intersect_subjects=False, check_validity=False
)
education_easy_pairing.describe()

Pairing against education:
  0: RN vs. 1: MD
  16 0 examples vs. 12 1 examples.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [38]:
display_pairing = statistics.build_pairing(education_easy_pairing[1], 'displayType')
display_pairing.describe()

Pairing against displayType:
  0: minimal vs. 1: full
  6 0 vs. 1 pairs.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [39]:
statistics.test_gaze_duration_outcomes(display_pairing)

visitDuration_infant:
  mean diff = -5.915; stdev diff = 21.476
  Paired t-test:
    |diff| > 0: p = 0.565
    diff < 0: p = 0.282
    diff > 0: p = 0.718
visitDuration_warmerInstrumentPanel:
  mean diff = -1.306; stdev diff = 4.637
  Paired t-test:
    |diff| > 0: p = 0.556
    diff < 0: p = 0.278
    diff > 0: p = 0.722
visitDuration_fiO2Dial:
  mean diff = 3.706; stdev diff = 6.360
  Paired t-test:
    |diff| > 0: p = 0.249
    diff < 0: p = 0.875
   ~diff > 0: p = 0.125
visitDuration_spO2ReferenceTable:
  mean diff = 4.509; stdev diff = 5.870
  Paired t-test:
   ~|diff| > 0: p = 0.147
    diff < 0: p = 0.927
   *diff > 0: p = 0.073
visitDuration_monitorFull:
  mean diff = -0.558; stdev diff = 28.798
  Paired t-test:
    |diff| > 0: p = 0.967
    diff < 0: p = 0.484
    diff > 0: p = 0.516
visitDuration_monitorBlank:
  mean diff = 5.905; stdev diff = 3.934
  Paired t-test:
  **|diff| > 0: p = 0.020
    diff < 0: p = 0.990
  **diff > 0: p = 0.010
visitDuration_monitorApgarTimer:
  me

### MD, Hard Scenarios

In [40]:
education_hard_pairing = statistics.build_pairing(
    scenario_split[1], 'education',
    intersect_subjects=False, check_validity=False
)
education_hard_pairing.describe()

Pairing against education:
  0: RN vs. 1: MD
  16 0 vs. 1 pairs.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [41]:
display_pairing = statistics.build_pairing(education_hard_pairing[1], 'displayType')
display_pairing.describe()

Pairing against displayType:
  0: minimal vs. 1: full
  8 0 vs. 1 pairs.
  Paired t-test alternative hypotheses:
    Ha left-tailed (diff < 0): mean 0 - mean 1 < 0
    Ha two-tailed (|diff| > 0): mean 0 - mean 1 != 0
    Ha right-tailed (diff > 0): mean 0 - mean 1 > 0


In [42]:
statistics.test_gaze_duration_outcomes(display_pairing)

visitDuration_infant:
  mean diff = 20.955; stdev diff = 18.743
  Paired t-test:
  **|diff| > 0: p = 0.021
    diff < 0: p = 0.989
  **diff > 0: p = 0.011
visitDuration_warmerInstrumentPanel:
  mean diff = -3.102; stdev diff = 8.589
  Paired t-test:
    |diff| > 0: p = 0.371
   ~diff < 0: p = 0.186
    diff > 0: p = 0.814
visitDuration_fiO2Dial:
  mean diff = -0.222; stdev diff = 2.370
  Paired t-test:
    |diff| > 0: p = 0.811
    diff < 0: p = 0.406
    diff > 0: p = 0.594
visitDuration_spO2ReferenceTable:
  mean diff = 0.727; stdev diff = 1.821
  Paired t-test:
    |diff| > 0: p = 0.326
    diff < 0: p = 0.837
   ~diff > 0: p = 0.163
visitDuration_monitorFull:
  mean diff = -23.921; stdev diff = 20.579
  Paired t-test:
  **|diff| > 0: p = 0.018
  **diff < 0: p = 0.009
    diff > 0: p = 0.991
visitDuration_monitorBlank:
  mean diff = 1.245; stdev diff = 12.423
  Paired t-test:
    |diff| > 0: p = 0.799
    diff < 0: p = 0.601
    diff > 0: p = 0.399
visitDuration_monitorApgarTimer:
 